## Preprocessing of baselines' original files


In this notebook, we preprocess the original files to make it applicable in our subset. Since, we focus on gender and the subset of the pronouns, most modifications will concern selecting the appropriate evaluation subset and ensuring that there's a common structure to the dataframes, including 'sentence', 'template', etc.

In [ ]:
# directory where we place the original raw files from WinoBias and Winogender
ORIG_FILES = "../data"
# directory containing an intermediate version of the files
RAW_DIR = "../results-baselines"
# directory containing the final version of the files
# (the ones that are actually referred to in the evaluation scripts)
PREPROC_DIR = "../results-baselines/final-results"

import pandas as pd
import numpy as np
import os, json
os.makedirs(RAW_DIR, exist_ok=True)
os.makedirs(PREPROC_DIR, exist_ok=True)

# The notebooks folder should be at the same level as the code folder...
import sys; sys.path.append("../src")
from run_pipeline import parse_replace_placeholders

In [ ]:
with open("../configs/placeholders.json") as f:
    PLACEHOLDERS = json.load(f)
    
PLACEHOLDERS

### Winobias

Proposed by Zhao et al 2018, roughly around the same time as WinoGender, comprises two types of coreference resolution examples. The first type, called Type 1, concerns the examples whose pronoun disambiguation requires implicit world knowledge and has no cues in the syntax or semantics of the example. The second type, called Type 2, is the easier set of coreference resolution examples, since syntax and semantics can help disambiguate the correct pronoun.


**Note**: ~~We do not need to download both anti-stereotypical and stereotypical associations because they are "symmetrical". That is, replacing the pronoun with the opposite template, would result in the stereotypical association.~~(Edit: Actually, we do need to download every file, since in some cases, we will find ourselves with sentences using pronouns "her" and we won't know which male pronoun to replace it with. Instead of wasting ChatGPT resources running this, we will process every file and remove duplicates in the end.)

In [ ]:
import re, glob
from collections import defaultdict

WINOBIAS_REGEX = r"(?P<entity>\[.+?\]).+?(?P<pronoun>\[.+?\])"
WINOBIAS_PATTERN = re.compile(WINOBIAS_REGEX)

def get_words(sentence: str, pattern=WINOBIAS_PATTERN):
    match = pattern.search(sentence)
    
    attribute = match.group("entity")
    target = match.group("pronoun")
    return attribute, target

def read_winobias(path: str):
    results = defaultdict(list)
    with open(path) as f:
        for l in f.readlines():
            l = l.strip()
            l = re.sub(r"^[0-9]{1,3} ", "", l)
            
            attr, target = get_words(l)
            
            l = l.replace(attr, attr[1:-1])
            l = l.replace(target, target[1:-1])
            
            results["sentence"].append(l)
            results["word"].append(attr[1:-1])
            results["target_word"].append(target[1:-1])
            
            for expr in ("she", "her", "hers", "herself"):
                if target[1:-1].lower() == expr:
                    results["drop"].append(True) # mark female results to drop
                    break
            else:
                results["drop"].append(False)
            
    results = pd.DataFrame(results)
    # Drop female results
    results = results[~results["drop"]]
    
    # Add information about the original file
    filename = path.rpartition("/")[-1]
    results["filename"] = filename
    
    results["stereotype"] = "pro_stereotyped" in filename
    results["is_challenging"] = "type1" in filename
    results["is_dev"] =  ".dev" in filename
        
    return pd.DataFrame(results)


for SUFFIX in (".dev", ".test"):
    # List all filepaths in the directory
    FILEPATHS = glob.glob(f"{ORIG_FILES}/winobias-zhao-2018/*.txt{SUFFIX}")
    # Merge all the examples in dev, regardless of the type
    winobias = pd.concat([read_winobias(fp) for fp in FILEPATHS]).sort_values("sentence").reset_index(drop=True)
    # Parse the templates, creating a template and determining whether the necessary pronouns appear.
    winobias_has_pronoun, winobias_template =  parse_replace_placeholders(
        winobias["sentence"].values.tolist(),
        PLACEHOLDERS["gender_to_placeholder"],
    )
    # Add information to the original file
    winobias.insert(len(winobias.columns), "has_pronoun", winobias_has_pronoun)
    winobias.insert(len(winobias.columns), "template", winobias_template)
    assert winobias["has_pronoun"].all(), "Some templates did not have a pronoun replaced"
    winobias.to_csv(f"{RAW_DIR}/coref__Winobias__templates{SUFFIX}.csv")
winobias.head(2)

In [ ]:
df = pd.read_csv(f"{RAW_DIR}/coref__Winobias__templates.dev.csv", index_col=0)
# let's drop the article
df["word"] = df["word"].apply(lambda x: x.split()[-1]).apply(str.lower)
df.to_csv(f"{PREPROC_DIR}/coref__Winobias__templates.dev.csv")


df = pd.read_csv(f"{RAW_DIR}/coref__Winobias__templates.test.csv", index_col=0)
# let's drop the article
df["word"] = df["word"].apply(lambda x: x.split()[-1]).apply(str.lower)
df.to_csv(f"{PREPROC_DIR}/coref__Winobias__templates.test.csv")

### Winogender

In [ ]:
def canonical_sentid(sentid: str) -> str:
    """Given the sentid field in the original Winogender files, strip them."""
    for exp in (".male.txt", ".female.txt", ".neutral.txt"):
        if sentid.endswith(exp):
            return sentid[:-len(exp)]        
    return sentid

winogender = pd.read_csv(f"{ORIG_FILES}/winogender-rudinger-2018/all_sentences.csv")
winogender.insert(1, "example_id", winogender["sentid"].apply(canonical_sentid))
winogender.head(5)

In [ ]:
# Since the sentences are the same, only changing the completion, drop all but the first.
winogender_subset = winogender.groupby("example_id").head(1)
# Create template from each sentence using the placeholders
winogender_has_pronoun, winogender_template =  parse_replace_placeholders(
    winogender_subset["sentence"].values.tolist(),
    PLACEHOLDERS["gender_to_placeholder"],
)

# Create columns 'has_pronoun', 'template'
winogender_subset.insert(len(winogender_subset.columns), "has_pronoun", winogender_has_pronoun)
winogender_subset.insert(len(winogender_subset.columns), "template", winogender_template)
assert winogender_subset["has_pronoun"].all(), "Some templates did not have a pronoun replaced"
winogender_subset.head()

In [ ]:
# Store
winogender_subset.to_csv(f"{RAW_DIR}/coref__Winogender__templates.csv", index=None)

In [ ]:
df = pd.read_csv(f"{RAW_DIR}/coref__Winogender__templates.csv", index_col=0)
# let's derive the word column
def obtain_word_winogender(example_id):
    e1, e2, idx = example_id.split(".")
    return (e2 if idx == "1" else e1).lower()

df["word"] = df["example_id"].apply(obtain_word_winogender)
df.to_csv(f"{PREPROC_DIR}/coref__Winogender__templates.csv")

### Stereoset

Even though Stereoset has two target words as "herself" (which wouldn't be difficult to disambiguate), the templates themselves have some pronouns. We have to create placeholders for these different cases.

In [ ]:
df = pd.read_csv(f"{RAW_DIR}/lm__StereoSet_pronouns_only.csv")
df = df.drop_duplicates()
df.groupby("target_word").count()

In [ ]:
df["sentence"].nunique()

In [ ]:
# We need to update the templates in herself
# -- since they are few, i'll do it manually :(
# -- we fixed the grammar on one example with stereoset id 3d2572d81efcfdb0829aea957f238b3f
df.drop("target_word",axis=1).to_csv(f"{PREPROC_DIR}/lm__StereoSet_pronouns_only.csv")

In [ ]:
df = pd.read_csv(f"{PREPROC_DIR}/lm__StereoSet_pronouns_only.csv")
df = df.drop_duplicates()
len(df)

In [ ]:
df.to_csv(f"{PREPROC_DIR}/lm__StereoSet_pronouns_only.csv", index=None)

### CrowS pairs

We perform the following transformations to the original dataset:

1. Restrict to the bias_type 'gender'
2. Keep the examples with more 60% agreement in terms of the bias type of the examples.
3. Since the sentences are minimally edited in the demographics, we will also try to check for the presence of male pronouns. Note that our code by default looks for male pronouns he/his/him only. However, because these examples reflect binary gender modifications, if our method fails to detect male pronoun in 'sent_more' because it was a female completion, then we assume that the 'sent_less' (which would be the male completion) should have the male pronouns. 

**Note**: Not all the resulting sentences are equally likely under both genders, due to the presence of gendered lexicon, like in sentences "As a woman, {SHE/HE} felt empowered ...".

In [ ]:
RAW_CROWS = pd.read_csv(f"{BASE_DIR}/results-baselines/lm__CrowS.csv", index_col=0)
len(RAW_CROWS)

In [ ]:
import json, sys; sys.path.append("../code")
from run_pipeline import parse_replace_placeholders

# gender-bias subset
RAW_CROWS = RAW_CROWS[RAW_CROWS["bias_type"] == "gender"]

# keep examples w/ "good agreement"
annotations = RAW_CROWS["annotations"].apply(lambda x: [annot == ['gender'] for annot in eval(x)])
# Note: we want at least 4 annotations and 3 of them should agree
b = annotations.apply(lambda x: sum(x) / len(x) > 0.60)

sents_more = RAW_CROWS["sent_more"].values.tolist()
sents_less = RAW_CROWS["sent_less"].values.tolist()

# What happens is that these are 
has_pronoun_more, template_more = parse_replace_placeholders(sents_more, PLACEHOLDERS["gender_to_placeholder"])
has_pronoun_less, template_less = parse_replace_placeholders(sents_less, PLACEHOLDERS["gender_to_placeholder"])

mask = (np.array(has_pronoun_more) | np.array(has_pronoun_less))
RAW_CROWS[mask]